# Sistema de Transformación de Oraciones en Guaraní con RAG

## Proyecto Final - PLN e IA

**Objetivo:** Implementar un sistema capaz de transformar oraciones en guaraní según reglas gramaticales específicas, comparando el rendimiento de:
- 2 modelos de LLM (GPT-3.5 Turbo vs Claude 3.5 Sonnet)
- 2 estrategias: **Sin RAG** (conocimiento del modelo) vs **Con RAG** (recuperación de documentación gramatical)

## Dataset: AmericasNLP 2025
- **Task:** Transformación de oraciones en guaraní
- **Input:** `Source` (oración base) + `Change` (tipo de transformación)
- **Output:** `Target` (oración transformada)

**Ejemplo:**
```
Source: "Ore ndorombyai kuri"
Change: "TYPE:AFF" (convertir a afirmativo)
Target: "Ore rombyai kuri"
```

## Metodología
1. Cargar dataset AmericasNLP (train/dev/test)
2. Construir base de conocimiento (RAG) con gramática guaraní
3. Implementar sistema de transformación con y sin RAG
4. Evaluar con métricas objetivas (BLEU, accuracy, etc.)
5. Comparar resultados

## 1. Instalación de Dependencias

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Instalación de librerías
!pip install -q -U \
  langchain langchain-core langchain-community langchain-text-splitters \
  sentence-transformers faiss-cpu \
  requests python-dotenv pymupdf pandas \
  sacrebleu nltk

print("✓ Dependencias instaladas correctamente")

## 2. Importaciones y Configuración

In [ ]:
import os
import json
import pandas as pd
import requests
import fitz  # PyMuPDF
from typing import List, Dict, Any, Tuple
import time

# LangChain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

# Métricas
from sacrebleu.metrics import BLEU
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

print("✓ Importaciones completadas")

## 3. Configuración de API Keys

In [ ]:
# Configurar OpenRouter API Key
# Opción 1: Usar Colab Secrets (recomendado)
# Opción 2: Ingresar manualmente

if 'OPENROUTER_API_KEY' in globals():
    os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY
    print("✓ API Key cargada desde variable del kernel")
else:
    from getpass import getpass
    api_key = getpass("Ingresa tu OPENROUTER_API_KEY: ")
    os.environ["OPENROUTER_API_KEY"] = api_key
    print("✓ API Key configurada")

## 4. Descarga del Dataset AmericasNLP

Dataset oficial: https://github.com/AmericasNLP/americasnlp2025/tree/main/ST2_EducationalMaterials/data

In [ ]:
# URLs del dataset
BASE_URL = "https://raw.githubusercontent.com/AmericasNLP/americasnlp2025/main/ST2_EducationalMaterials/data/"

datasets_urls = {
    "train": f"{BASE_URL}guarani-train.tsv",
    "dev": f"{BASE_URL}guarani-dev.tsv",
    "test": f"{BASE_URL}guarani-test.tsv"
}

def load_dataset(url: str) -> pd.DataFrame:
    """Carga el dataset desde URL"""
    response = requests.get(url)
    response.raise_for_status()
    
    # Guardar temporalmente
    filename = url.split("/")[-1]
    with open(filename, "w", encoding="utf-8") as f:
        f.write(response.text)
    
    # Leer con pandas
    df = pd.read_csv(filename, sep="\t")
    return df

# Cargar datasets
print("Descargando datasets...\n")
datasets = {}

for split, url in datasets_urls.items():
    try:
        df = load_dataset(url)
        datasets[split] = df
        print(f"✓ {split.upper()}: {len(df)} ejemplos")
        print(f"  Columnas: {list(df.columns)}")
    except Exception as e:
        print(f"❌ Error cargando {split}: {e}")

# Vista previa
print("\n" + "="*60)
print("EJEMPLO DEL DATASET (primeras 3 filas de DEV):")
print("="*60)
print(datasets["dev"].head(3))

## 5. Carga de Gramática Guaraní (Base de Conocimiento para RAG)

**Nota:** Sube el archivo `Gramática guaraní.pdf` a Colab o usa Google Drive.

In [ ]:
# Opción 1: Subir archivo manualmente
from google.colab import files
print("Por favor, sube el archivo 'Gramática guaraní.pdf'")
uploaded = files.upload()
PDF_PATH = list(uploaded.keys())[0]

# Opción 2: Desde Google Drive (descomentar si prefieres esta opción)
# from google.colab import drive
# drive.mount('/content/drive')
# PDF_PATH = '/content/drive/MyDrive/Gramática guaraní.pdf'

In [ ]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """Extrae texto del PDF de gramática"""
    doc = fitz.open(pdf_path)
    text = ""
    
    print(f"📘 Extrayendo texto de: {pdf_path}")
    print(f"   Total de páginas: {len(doc)}")
    
    for page_num, page in enumerate(doc, start=1):
        page_text = page.get_text()
        text += f"\n--- Página {page_num} ---\n{page_text}"
        
        if page_num % 10 == 0:
            print(f"   • Procesadas: {page_num} páginas")
    
    doc.close()
    print(f"✓ Extracción completada: {len(text)} caracteres")
    return text.strip()

# Extraer texto
pdf_text = extract_text_from_pdf(PDF_PATH)

# Vista previa
print("\n" + "="*60)
print("MUESTRA DEL TEXTO (primeros 500 caracteres):")
print("="*60)
print(pdf_text[:500] + "...")

## 6. Construcción del Vector Store (RAG)

In [ ]:
# Dividir texto en chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

print("📚 Dividiendo texto en chunks...")
text_chunks = text_splitter.split_text(pdf_text)

# Crear documentos
documents = [
    Document(
        page_content=chunk,
        metadata={"source": "Gramática guaraní", "chunk_id": idx}
    )
    for idx, chunk in enumerate(text_chunks)
]

print(f"✓ Creados {len(documents)} chunks")
print(f"  Tamaño promedio: {sum(len(c) for c in text_chunks) / len(text_chunks):.0f} caracteres")

In [ ]:
# Cargar modelo de embeddings
print("🔄 Cargando modelo de embeddings...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)
print("✓ Modelo de embeddings listo")

In [ ]:
# Crear vector store con FAISS
print("💾 Creando vector store...")
vectorstore = FAISS.from_documents(documents, embeddings)

# Configurar retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Top 3 documentos más relevantes
)

print("✓ Vector store creado")
print("✓ Retriever configurado (k=3)")

## 7. Wrapper para OpenRouter (GPT-3.5 y Claude 3.5)

In [ ]:
class OpenRouterLLM:
    """Wrapper para usar modelos via OpenRouter"""
    
    def __init__(self, model_name: str, api_key: str):
        self.model_name = model_name
        self.api_key = api_key
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
    
    def generate(self, prompt: str, max_tokens: int = 200, temperature: float = 0.3) -> str:
        """Genera respuesta del modelo"""
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://colab.research.google.com/",
            "X-Title": "guarani-transformation"
        }
        
        data = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": max_tokens,
            "temperature": temperature
        }
        
        try:
            response = requests.post(self.api_url, headers=headers, json=data, timeout=60)
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"].strip()
            else:
                return f"Error {response.status_code}: {response.text}"
        except Exception as e:
            return f"Error: {str(e)}"

# Configurar modelos
api_key = os.environ.get("OPENROUTER_API_KEY")

model_gpt35 = OpenRouterLLM("openai/gpt-3.5-turbo", api_key)
model_claude = OpenRouterLLM("anthropic/claude-3.5-sonnet", api_key)

print("✓ Modelos configurados:")
print("  - GPT-3.5 Turbo (OpenAI)")
print("  - Claude 3.5 Sonnet (Anthropic)")

## 8. Sistema de Transformación de Oraciones

In [ ]:
class GuaraniTransformationSystem:
    """Sistema de transformación de oraciones en guaraní"""
    
    def __init__(self, llm, retriever=None):
        self.llm = llm
        self.retriever = retriever
    
    def transform_without_rag(self, source: str, change: str) -> str:
        """Transformación SIN RAG (solo conocimiento del modelo)"""
        prompt = f"""Eres un experto en el idioma guaraní.

Transforma la siguiente oración en guaraní según la regla indicada.

Oración original: "{source}"
Regla de transformación: {change}

Responde ÚNICAMENTE con la oración transformada en guaraní, sin explicaciones adicionales.

Oración transformada:"""
        
        return self.llm.generate(prompt)
    
    def transform_with_rag(self, source: str, change: str) -> str:
        """Transformación CON RAG (con contexto gramatical)"""
        # Recuperar contexto relevante
        query = f"transformación {change} en guaraní negación afirmación"
        docs = self.retriever.invoke(query)
        context = "\n\n".join([d.page_content[:600] for d in docs[:3]])
        
        prompt = f"""Eres un experto en el idioma guaraní.

Usa la siguiente información gramatical para transformar la oración:

CONTEXTO GRAMATICAL:
{context}

TAREA:
Oración original: "{source}"
Regla de transformación: {change}

INSTRUCCIONES:
- Basa tu transformación en las reglas gramaticales del contexto
- Responde ÚNICAMENTE con la oración transformada en guaraní
- NO agregues explicaciones

Oración transformada:"""
        
        return self.llm.generate(prompt)
    
    def evaluate_dataset(self, df: pd.DataFrame, use_rag: bool = False) -> List[Dict]:
        """Evalúa el sistema en un dataset completo"""
        results = []
        
        for idx, row in df.iterrows():
            source = row["Source"]
            change = row["Change"]
            target = row["Target"]
            
            # Generar predicción
            if use_rag:
                prediction = self.transform_with_rag(source, change)
            else:
                prediction = self.transform_without_rag(source, change)
            
            results.append({
                "id": row["ID"],
                "source": source,
                "change": change,
                "target": target,
                "prediction": prediction,
                "correct": prediction.strip().lower() == target.strip().lower()
            })
            
            # Pequeña pausa para no saturar la API
            time.sleep(0.5)
        
        return results

print("✓ Sistema de transformación implementado")

## 9. Evaluación de Modelos

Se evaluarán ambos modelos (GPT-3.5 y Claude 3.5) con y sin RAG.

In [ ]:
# Evaluación en DEV set (usamos DEV primero para ajustar, luego TEST para resultados finales)
dev_data = datasets["dev"].head(10)  # Primero probamos con 10 ejemplos

print("="*70)
print("EVALUANDO MODELO: GPT-3.5 TURBO SIN RAG")
print("="*70)

system_gpt_no_rag = GuaraniTransformationSystem(model_gpt35, retriever=None)
results_gpt_no_rag = system_gpt_no_rag.evaluate_dataset(dev_data, use_rag=False)

print("\n" + "="*70)
print("EVALUANDO MODELO: GPT-3.5 TURBO CON RAG")
print("="*70)

system_gpt_rag = GuaraniTransformationSystem(model_gpt35, retriever=retriever)
results_gpt_rag = system_gpt_rag.evaluate_dataset(dev_data, use_rag=True)

print("\n" + "="*70)
print("EVALUANDO MODELO: CLAUDE 3.5 SONNET SIN RAG")
print("="*70)

system_claude_no_rag = GuaraniTransformationSystem(model_claude, retriever=None)
results_claude_no_rag = system_claude_no_rag.evaluate_dataset(dev_data, use_rag=False)

print("\n" + "="*70)
print("EVALUANDO MODELO: CLAUDE 3.5 SONNET CON RAG")
print("="*70)

system_claude_rag = GuaraniTransformationSystem(model_claude, retriever=retriever)
results_claude_rag = system_claude_rag.evaluate_dataset(dev_data, use_rag=True)

print("\n✓ Evaluación completada")

## 10. Cálculo de Métricas

In [ ]:
def calculate_metrics(results: List[Dict]) -> Dict:
    """Calcula métricas de evaluación"""
    total = len(results)
    correct = sum(1 for r in results if r["correct"])
    accuracy = (correct / total) * 100 if total > 0 else 0
    
    # BLEU score
    bleu = BLEU()
    references = [[r["target"]] for r in results]
    predictions = [r["prediction"] for r in results]
    bleu_score = bleu.corpus_score(predictions, references).score
    
    return {
        "total": total,
        "correct": correct,
        "accuracy": accuracy,
        "bleu": bleu_score
    }

# Calcular métricas para cada configuración
metrics = {
    "GPT-3.5 Sin RAG": calculate_metrics(results_gpt_no_rag),
    "GPT-3.5 Con RAG": calculate_metrics(results_gpt_rag),
    "Claude 3.5 Sin RAG": calculate_metrics(results_claude_no_rag),
    "Claude 3.5 Con RAG": calculate_metrics(results_claude_rag)
}

# Mostrar resultados
print("\n" + "="*70)
print("RESULTADOS DE EVALUACIÓN")
print("="*70)

for config, metrics_data in metrics.items():
    print(f"\n{config}:")
    print(f"  Accuracy: {metrics_data['accuracy']:.2f}%")
    print(f"  BLEU Score: {metrics_data['bleu']:.2f}")
    print(f"  Correctas: {metrics_data['correct']}/{metrics_data['total']}")

## 11. Tabla Comparativa

In [ ]:
# Crear DataFrame comparativo
comparison_df = pd.DataFrame({
    "Modelo": ["GPT-3.5 Turbo", "GPT-3.5 Turbo", "Claude 3.5 Sonnet", "Claude 3.5 Sonnet"],
    "Estrategia": ["Sin RAG", "Con RAG", "Sin RAG", "Con RAG"],
    "Accuracy (%)": [
        metrics["GPT-3.5 Sin RAG"]["accuracy"],
        metrics["GPT-3.5 Con RAG"]["accuracy"],
        metrics["Claude 3.5 Sin RAG"]["accuracy"],
        metrics["Claude 3.5 Con RAG"]["accuracy"]
    ],
    "BLEU Score": [
        metrics["GPT-3.5 Sin RAG"]["bleu"],
        metrics["GPT-3.5 Con RAG"]["bleu"],
        metrics["Claude 3.5 Sin RAG"]["bleu"],
        metrics["Claude 3.5 Con RAG"]["bleu"]
    ]
})

print("\n" + "="*70)
print("TABLA COMPARATIVA")
print("="*70)
print(comparison_df.to_string(index=False))

# Encontrar mejor configuración
best_idx = comparison_df["Accuracy (%)"].idxmax()
best_config = comparison_df.iloc[best_idx]

print("\n" + "="*70)
print("MEJOR CONFIGURACIÓN")
print("="*70)
print(f"Modelo: {best_config['Modelo']}")
print(f"Estrategia: {best_config['Estrategia']}")
print(f"Accuracy: {best_config['Accuracy (%)']:.2f}%")
print(f"BLEU: {best_config['BLEU Score']:.2f}")

## 12. Ejemplos de Transformaciones

In [ ]:
# Mostrar algunos ejemplos de transformaciones
print("\n" + "="*70)
print("EJEMPLOS DE TRANSFORMACIONES (GPT-3.5 CON RAG)")
print("="*70)

for i, result in enumerate(results_gpt_rag[:5], 1):
    print(f"\nEjemplo {i}:")
    print(f"  ID: {result['id']}")
    print(f"  Oración original: {result['source']}")
    print(f"  Transformación: {result['change']}")
    print(f"  Esperado: {result['target']}")
    print(f"  Generado: {result['prediction']}")
    print(f"  ✓ Correcto" if result['correct'] else "  ✗ Incorrecto")

## 13. Exportar Resultados

In [ ]:
# Guardar resultados completos
all_results = {
    "gpt35_no_rag": results_gpt_no_rag,
    "gpt35_rag": results_gpt_rag,
    "claude_no_rag": results_claude_no_rag,
    "claude_rag": results_claude_rag,
    "metrics": metrics,
    "metadata": {
        "dataset": "AmericasNLP 2025 - Guaraní",
        "models": ["openai/gpt-3.5-turbo", "anthropic/claude-3.5-sonnet"],
        "strategies": ["Sin RAG", "Con RAG"],
        "embedding_model": "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
        "vector_store": "FAISS",
        "grammar_source": "Gramática guaraní.pdf"
    }
}

# Guardar en JSON
with open("guarani_transformation_results.json", "w", encoding="utf-8") as f:
    json.dump(all_results, f, ensure_ascii=False, indent=2)

print("✓ Resultados guardados en: guarani_transformation_results.json")

# Guardar tabla comparativa en CSV
comparison_df.to_csv("comparison_table.csv", index=False)
print("✓ Tabla comparativa guardada en: comparison_table.csv")

# Descargar archivos (solo en Colab)
try:
    from google.colab import files
    files.download("guarani_transformation_results.json")
    files.download("comparison_table.csv")
    print("\n✓ Archivos descargados")
except:
    print("\n⚠️ No estás en Colab. Archivos guardados localmente.")

## 14. Conclusiones

### Preguntas Clave a Responder:

1. **¿Qué modelo es mejor?**
   - Analizar accuracy y BLEU score
   - Considerar velocidad y costo

2. **¿RAG mejora el rendimiento?**
   - Comparar métricas con/sin RAG
   - Analizar ejemplos donde RAG ayuda

3. **¿Por qué estos resultados?**
   - Guaraní es idioma de bajo recursos
   - RAG aporta conocimiento gramatical específico
   - Modelos grandes (Claude) vs rápidos (GPT-3.5)

### Próximos Pasos:
- Evaluar en TEST set completo
- Ajustar prompts según resultados
- Considerar fine-tuning si hay datos suficientes
- Agregar más documentación gramatical al vector store